[Reference](https://medium.com/@sharlene553293/forecasting-seattle-travel-and-hotel-occupancy-with-arima-2c5c3e790b9c)

# Step 1: Cleaning Data - Airport Traffic as a Demand Proxy

In [1]:
# Reading and cleaning the data
import pandas as pd
data = pd.read_csv('tsa_screened.csv')
# creating week start date for plotting and training later
data['week_start'] = pd.to_datetime(data['year'].astype(str) + '-W' + data['week'].astype(str) + '-1', format='%Y-W%W-%w')
# Convert 'total_screened' to numeric, handling commas
data['total_screened'] = data['total_screened'].str.replace(',', '').astype(int)

import matplotlib.pyplot as plt
import seaborn as sns

mini_data = data[data['year']>=2021]

plt.figure(figsize=(12, 6))
sns.lineplot(data=mini_data, x='week_start', y='total_screened')
plt.title('Total Screened Over Time')
plt.xlabel('Week Start Date')
plt.ylabel('Total Screened')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Step 2: Testing for Stationarity

There are two tests we can do here
- ADF test: checks for unit roots (non-stationarity).
- KPSS test: checks for trend-stationarity.

In [2]:
from statsmodels.tsa.stattools import adfuller, kpss

# Create weekly time series index
y = mini_data['total_screened'].values
n = len(y)
ts = pd.Series(y, index=pd.date_range("2021-01-04", periods=n, freq="W"))

# 1. Augmented Dickey-Fuller Test (ADF)
adf_result = adfuller(ts)
print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])
print("Critical Values:", adf_result[4])

# 2. KPSS Test
kpss_result = kpss(ts, regression="c", nlags="auto")
print("\nKPSS Statistic:", kpss_result[0])
print("p-value:", kpss_result[1])
print("Critical Values:", kpss_result[3])

# Step 3: Fitting the Forecast Models

In [3]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Put into pandas series with weekly datetime index
y = mini_data['total_screened'].values
n = len(y)
ts = pd.Series(y, index=pd.date_range("2021-01-04", periods=n, freq="W"))

# Plot ACF and PACF
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ACF
plot_acf(ts, lags=52, ax=axes[0])   # up to 52 lags (1 year if weekly)
axes[0].set_title("Autocorrelation Function (ACF)")

# PACF
plot_pacf(ts, lags=52, ax=axes[1], method="ywm")  # "ywm" is stable for PACF
axes[1].set_title("Partial Autocorrelation Function (PACF)")

plt.tight_layout()
plt.show()

In [4]:
ts_diff = ts.diff().dropna()
# Plot ACF and PACF of differenced series
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ACF
plot_acf(ts_diff, lags=52, ax=axes[0])   # up to 52 lags (1 year if weekly)
axes[0].set_title("Autocorrelation Function (ACF) of Differenced Series")

# PACF
plot_pacf(ts_diff, lags=52, ax=axes[1], method="ywm")  # "ywm" is stable for PACF
axes[1].set_title("Partial Autocorrelation Function (PACF) of Differenced Series")

plt.tight_layout()
plt.show()ts_diff = ts.diff().dropna()
# Plot ACF and PACF of differenced series
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ACF
plot_acf(ts_diff, lags=52, ax=axes[0])   # up to 52 lags (1 year if weekly)
axes[0].set_title("Autocorrelation Function (ACF) of Differenced Series")

# PACF
plot_pacf(ts_diff, lags=52, ax=axes[1], method="ywm")  # "ywm" is stable for PACF
axes[1].set_title("Partial Autocorrelation Function (PACF) of Differenced Series")

plt.tight_layout()
plt.show()

In [5]:
import pmdarima as pm

# Fit auto ARIMA (like R's auto.arima)
model = pm.auto_arima(
    ts,
    seasonal=True,          # SARIMA
    m=52,                   # seasonality period (52 weeks ~ yearly)
    stepwise=True,          # stepwise search like R
    suppress_warnings=True,
    trace=True
)

print(model.summary())

# Forecast next 10 weeks
n_forecast = 10
forecast = model.predict(n_periods=n_forecast)

# Plot forecast vs. history
plt.figure(figsize=(12, 6))
plt.plot(ts, label="History")
future_index = pd.date_range(ts.index[-1] + pd.Timedelta(weeks=1),
                             periods=n_forecast, freq="W")
plt.plot(future_index, forecast, label="Forecast", color="red")
plt.legend()
plt.show()

In [6]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# Get residuals from your fitted ARIMA model
# fit = loaded_model.fit(ts)
# resid = fit.resid
res_id = loaded_model.resid()

# Plot ACF of residuals
fig, ax = plt.subplots(figsize=(10,5))
plot_acf(res_id, lags=104, ax=ax)  # up to 2 years of weekly lags

# Add vertical lines at seasonal multiples
seasonal_period = 52
for lag in [seasonal_period, 2*seasonal_period]:
    plt.axvline(x=lag, color='red', linestyle='--', alpha=0.7)

plt.title("Residual ACF with seasonal lags marked")
plt.show()

## Step 3a. Fitting ETS Model

In [7]:
#ETS  https://otexts.com/fpp2/ets-forecasting.html
import numpy as np
import pandas as pd
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

def fit_ets_auto(y, seasonal_periods=None):
    # Candidate options (like R's ets())
    error_types = ['add', 'mul']
    trend_types = [None, 'add', 'mul']
    seasonal_types = [None, 'add', 'mul']

    best_aicc = np.inf
    best_model = None
    best_params = None

    for error in error_types:
        for trend in trend_types:
            for seasonal in seasonal_types:
                # Skip invalid combinations (like multiplicative without seasonality)
                if seasonal is None and seasonal_periods is not None:
                    continue

                try:
                    model = ETSModel(
                        y,
                        error=error,
                        trend=trend,
                        seasonal=seasonal,
                        seasonal_periods=seasonal_periods
                    )
                    fit = model.fit(disp=False)

                    # Compute AICc = AIC + [2k(k+1)] / (n-k-1)
                    n = len(y)
                    k = fit.params.shape[0]
                    aic = fit.aic
                    aicc = aic + (2*k*(k+1)) / (n-k-1)

                    if aicc < best_aicc:
                        best_aicc = aicc
                        best_model = fit
                        best_params = (error, trend, seasonal)
                except Exception as e:
                    continue

    print(f"Best ETS model: error={best_params[0]}, trend={best_params[1]}, seasonal={best_params[2]}, AICc={best_aicc:.2f}")
    return best_model

# Example usage
# y = your time series as a pandas Series
ets_fit = fit_ets_auto(y, seasonal_periods=52)
print(ets_fit.summary())

In [8]:
import matplotlib.pyplot as plt
import pandas as pd

# Forecast horizon (52 weeks ahead)
forecast_steps = 52

# Forecast future values (returns ndarray)
forecast_array = ets_fit.forecast(steps=forecast_steps)


# Build future index (assuming daily data; change "D" to "W", "M", etc.)
future_index = pd.date_range(ts.index[-1] + pd.Timedelta(weeks=1),
                             periods=forecast_steps, freq="W")

# Plot
plt.figure(figsize=(12, 6))
plt.plot(ts.index, y, label="Actual", color="blue")

# plt.plot(future_index, forecast, label="Forecast", color="red")
plt.plot(future_index, forecast_array, label="Forecast", color="green", linestyle=":")

plt.xlabel("Time")
plt.ylabel("Values")
plt.title("Actual, Predicted, and Forecasted")
plt.legend()
plt.show()

## Step 3b. Fitting a LSTM Model

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# ==== 1. Prepare data ====
values = mini_data["total_screened"].values.reshape(-1, 1)  # (n, 1)

# Scale
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(values)

# Sequence length (52 weeks = 1 year seasonality)
seq_length = 52
X, y = [], []
for i in range(len(scaled_values) - seq_length):
    X.append(scaled_values[i:i+seq_length])
    y.append(scaled_values[i+seq_length])

X = np.array(X)  # (samples, seq_length, 1)
y = np.array(y)  # (samples, 1)


# Reshape for LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))  # (samples, timesteps, features)

# Train/test split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# ==== 2. Build model ====
model = Sequential([
    LSTM(64, activation='tanh', return_sequences=False, input_shape=(seq_length, 2)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# ==== 3. Train ====
history = model.fit(X_train, y_train, epochs=50, batch_size=16,
                    validation_data=(X_test, y_test), verbose=1)

# ==== 4. Forecast recursively ====
n_forecast = 52
last_seq_values = scaled_values[-seq_length:]
last_seq_week = week_of_year[-seq_length:] / 52.0
last_sequence = np.hstack([last_seq_values, last_seq_week]).reshape(1, seq_length, 2)

# ==== Forecast loop ====
forecast_scaled = []
current_seq = last_sequence.copy()

for i in range(n_forecast):
    pred = model.predict(current_seq, verbose=0)[0, 0]
    forecast_scaled.append(pred)

    # Compute next week-of-year as float between 0-1
    next_week = float(((week_of_year[-seq_length + i + 1] if i+1 < len(week_of_year) else (week_of_year[-1]+i+1)) % 52) / 52.0)

    # Combine predicted value and week-of-year
    next_feature = np.array([[float(pred), next_week]], dtype=float)

    # Append to sequence
    current_seq = np.append(current_seq[:, 1:, :], next_feature.reshape(1, 1, 2), axis=1)

# Inverse scale
forecast = scaler.inverse_transform(np.array(forecast_scaled).reshape(-1, 1))

# ==== 5. Plot ====
plt.figure(figsize=(12,6))
plt.plot(mn.index, mn["total_screened"], label="History")
future_index = pd.date_range(start=mn.index[-1] + pd.Timedelta(weeks=1),
                             periods=n_forecast, freq="W")
plt.plot(future_index, forecast, label="Forecast", color="red")
plt.legend()
plt.title("LSTM Forecast with 52-week sequence + week-of-year")
plt.show()aa

# Step 4: Linking Arrivals to Occupancy

In [10]:
q1_data = data[data['week']<=12]
q1_data['total_screened'] = q1_data['total_screened'].astype(float)
q1_data[['year','total_screened']].groupby('year').mean()

In [11]:
df = pd.DataFrame({
    "year": [2021, 2022, 2023, 2024, 2025],
    "arrivals": [18566.666667, 35100.000000, 43166.666667, 45191.666667, 46158.333333],   # millions
    "occupancy": [0.326, 0.558, 0.675, 0.683, 0.703]  # proportion
})

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# ========== 1. Simple Ratio / Scaling ==========
df['arrivals'] = df['arrivals'] / 1000000  # convert to millions
df["ratio"] = df["occupancy"] / df["arrivals"]
avg_ratio = df["ratio"].mean()
df["occ_ratio_pred"] = df["arrivals"] * avg_ratio

In [12]:
# ========== 2. Constrained Linear Regression (intercept = 0) ==========
# y = beta * x
X = df["arrivals"].values.reshape(-1,1)
y = df["occupancy"].values

# force intercept=0 by not adding constant
model_constrained = sm.OLS(y, X).fit()
df["occ_lin0_pred"] = model_constrained.predict(X)

The logistic curve proved most realistic: as arrivals increase, occupancy rises, but never exceeds 100%.

In [13]:
# ========== 3. Unconstrained Linear Regression ==========
Xc = sm.add_constant(df["arrivals"])  # adds intercept
model_lin = sm.OLS(y, Xc).fit()
df["occ_lin_pred"] = model_lin.predict(Xc)

In [14]:
# ========== Plot ==========
plt.figure(figsize=(8,6))
plt.scatter(df["arrivals"], df["occupancy"], label="Observed", c="black")

plt.plot(df["arrivals"], df["occ_ratio_pred"], "--", label=f"Ratio avg method ({avg_ratio:.3f})")
plt.plot(df["arrivals"], df["occ_lin0_pred"], "--", label=f"Linear (no intercept) slope={model_constrained.params[0]:.3f}")
plt.plot(df["arrivals"], df["occ_lin_pred"], "--", label=f"Linear w/ intercept slope={model_lin.params[1]:.3f}")

plt.xlabel("Airport Arrivals (millions, Q1)")
plt.ylabel("Occupancy Rate")
plt.title("Mapping Airport Arrivals → Occupancy")
plt.legend()
plt.show()


# ========== Results Table ==========
print(df[["year", "arrivals", "occupancy", "occ_ratio_pred", "occ_lin0_pred", "occ_lin_pred"]])

In [15]:
#forecast
forecast_df = pd.read_csv('tsa_screen_forecast.csv')
forecast_df['forecast']=forecast_df['forecast']/1000000

In [16]:
# ---- 1. Ratio Method ----
forecast_df["occ_ratio_pred"] = forecast_df["forecast"] * avg_ratio

# ---- 2. Constrained Linear Regression (no intercept) ----
forecast_df["occ_lin0_pred"] = model_constrained.predict(forecast_df["forecast"].values.reshape(-1,1))

# ---- 3. Unconstrained Linear Regression ----
X_future = sm.add_constant(forecast_df["forecast"])  # add intercept
forecast_df["occ_lin_pred"] = model_lin.predict(X_future)

# ---- Optional: clip to realistic bounds ----
forecast_df["occ_clipped"] = forecast_df["occ_lin_pred"].clip(lower=0.3, upper=0.9)

plt.figure(figsize=(12,6))

# Plot forecasts (as lines, minimal markers)
plt.plot(forecast_df["week_start"], forecast_df["occ_ratio_pred"], label="Forecast: Ratio method", linewidth=2)
plt.plot(forecast_df["week_start"], forecast_df["occ_lin0_pred"], label="Forecast: Linear no intercept", linewidth=2)
plt.plot(forecast_df["week_start"], forecast_df["occ_lin_pred"], label="Forecast: Linear w/ intercept", linewidth=2, linestyle="--")
# plt.plot(forecast_df["week_start"], forecast_df["occ_clipped"], label="Forecast: Linear (clipped realistic)", linewidth=2, linestyle=":")

plt.xlabel("Time (Week)")
plt.ylabel("Predicted Occupancy Rate")
plt.title("Forecasted Occupancy from Airport Arrivals")

# Rotate labels & reduce clutter
plt.xticks(rotation=45)
if len(forecast_df) > 20:  # weekly-level
    plt.xticks(forecast_df["week_start"][::4])  # show every 4th tick to declutter

plt.legend()
plt.tight_layout()
plt.show()

In [17]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Logistic function
def logistic(x, L, k, x0):
    return L / (1 + np.exp(-k * (x - x0)))

# Fit logistic to observed data
xdata = df["arrivals"].values
ydata = df["occupancy"].values

# Initial guesses: L=1 (100% cap), k=1 (steepness), x0=mean arrivals
p0 = [1.0, 1.0, np.mean(xdata)]
params, _ = curve_fit(logistic, xdata, ydata, p0=p0, maxfev=10000)
L, k, x0 = params

print(f"Logistic fit params: L={L:.3f}, k={k:.3f}, x0={x0:.3f}")

forecast_df['week_start_'] = pd.to_datetime(forecast_df['week_start'])

# Predictions for observed + forecast
mini_data['arrivals'] = mini_data['total_screened'] / 1000000  # convert to millions
mini_data["occ_logistic_pred"] = logistic(mini_data["arrivals"], L, k, x0)
forecast_df["occ_logistic_pred"] = logistic(forecast_df["forecast"], L, k, x0)

plt.figure(figsize=(12,6))

# Observed (fitted logistic curve over time)
plt.plot(mini_data["week_start"], mini_data["occ_logistic_pred"], "r-", label="Logistic fit (observed)")

# Forecast (logistic projection over time)
plt.plot(forecast_df["week_start_"], forecast_df["occ_logistic_pred"], "b--", label="Logistic forecast")

# # Actual observed occupancy (if you want to show points)
# plt.scatter(mini_data["week_start"], mini_data["occupancy"], color="black", alpha=0.7, label="Observed data")

# Formatting
plt.xlabel("Week Start")
plt.ylabel("Occupancy Rate")
plt.title("Observed vs Forecasted Occupancy (Logistic Fit)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()